<a href="https://colab.research.google.com/github/Guiils/analise_de_dados/blob/main/Tratamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymongo

In [ ]:
pip install gcsfs

In [ ]:
pip install pandera

In [ ]:
import pymongo
from pymongo import MongoClient

In [ ]:
import pandera as pa

In [ ]:
import pandas as pd
import numpy as np
import os
from google.cloud import storage

In [ ]:
#Conector MongoDB Atlas
uri = "mongodb+srv://clusterad1.zicdsgn.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,tls=True,tlsCertificateKeyFile='/content/X509-cert-4581822443265424952.pem')

db = client['cenipa']
colecao = db['tratados']

In [ ]:
serviceAccount = '/content/advance-airline-377819-1bf613aac07c.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
client = storage.Client()
bucket = client.get_bucket('ad1-bucket')
bucket.blob('ocorrenciass.csv')
path = 'gs://ad1-bucket/brutos/ocorrenciass.csv'

##Cenipa

bruto

In [ ]:
df = pd.read_csv(path,sep=';',encoding='ISO-8859-1',parse_dates=['ocorrencia_dia'],dayfirst=True)

In [ ]:
db = client['bruto']
colecaocenipa = db['cenipa']
colecaocenipa.count_documents({})

6769

In [ ]:
df_dict = df.to_dict("records")

colecaocenipa.insert_many(df_dict)

In [ ]:
dfback = df.copy()

tratado

In [ ]:
# Drop de colunas identicas ( codigo_ocorrencia1 , codigo_ocorrencia2,codigo_ocorrencia3 , codigo_ocorrencia4)
df.drop(['codigo_ocorrencia1','codigo_ocorrencia2','codigo_ocorrencia3','codigo_ocorrencia4'],axis=1,inplace=True)
# Drop de colunas com problemas e/ou não utilizadas na analise
df.drop(['ocorrencia_latitude','ocorrencia_longitude','investigacao_aeronave_liberada','investigacao_status','divulgacao_relatorio_numero','divulgacao_relatorio_publicado','divulgacao_dia_publicacao'],axis=1,inplace=True)

In [ ]:
#Renomeando colunas
df.rename(columns={'ocorrencia_classificacao':'classificacao','ocorrencia_cidade':'cidade','ocorrencia_uf':'uf','ocorrencia_pais':'pais'},inplace=True)
df.rename(columns={'ocorrencia_aerodromo':'aerodromo','ocorrencia_dia':'data','ocorrencia_hora':'horario','total_aeronaves_envolvidas':'n_aeronaves','ocorrencia_saida_pista':'saida_pista'},inplace=True)

In [ ]:
df.replace(['***','**NI'],pd.NA,inplace=True)

In [ ]:
df.replace(pd.NA,np.NaN,inplace=True)

In [ ]:
#Converter coluna data para string
df['data'] = df['data'].astype(str)

In [ ]:
#Criar coluna periodo (data + horario)
df['periodo'] = pd.to_datetime(df['data']+' '+df['horario'])

In [ ]:
#Criação do schema pandera

schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'classificacao':pa.Column(pa.String),
        'cidade':pa.Column(pa.String),
        'uf':pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        'pais':pa.Column(pa.String),
        'aerodromo':pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
        'data':pa.Column(pa.DateTime),
        'horario':pa.Column(pa.String,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'),nullable=True),
        'total_recomendacoes':pa.Column(pa.Int),
        'n_aeronaves':pa.Column(pa.Int),
        'saida_pista':pa.Column(pa.String,pa.Check.str_length(3,3)),
    }
)

schema2 = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia':pa.Column(pa.Int),
        'classificacao':pa.Column(pa.String),
        'cidade':pa.Column(pa.String),
        'uf':pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        'pais':pa.Column(pa.String),
        'aerodromo':pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
        'total_recomendacoes':pa.Column(pa.Int),
        'n_aeronaves':pa.Column(pa.Int),
        'saida_pista':pa.Column(pa.String,pa.Check.str_length(3,3)),
        'periodo':pa.Column(pa.DateTime,nullable=True),
    }
)

In [ ]:
df.drop(['data','horario'],axis=1,inplace=True)

In [ ]:
df['periodo'] = df['periodo'].astype(str)

In [ ]:
db = client['tratado']
colecaocenipa2 = db['cenipa']


In [ ]:
df_dict2 = df.to_dict("records")

colecaocenipa2.insert_many(df_dict2)

## Airbnb

Brutos

In [ ]:
path2 = 'gs://ad1-bucket/brutos/amsterdam_weekdays.csv'

In [ ]:
df2 = pd.read_csv(path2)

In [ ]:
db = client['bruto']
colecaoairbnb = db['airbnb']

In [ ]:
df2_dict = df2.to_dict("records")

colecaoairbnb.insert_many(df2_dict)

Tratados

In [ ]:
df2.drop(['Unnamed: 0'], axis=1, inplace = True)

In [ ]:
df2.rename(columns={'room_type':'tipo', 'room_shared':'compartilhado','room_private':'privado','person_capacity':'p_capacidade','host_is_superhost':'superhost','cleanliness_rating':'nota_limpeza','guest_satisfaction_overall':'media_satisfacao','bedrooms':'quartos'}, inplace=True)

In [ ]:
df2.loc[df2.tipo == 'Private room', ['tipo']] = 'Quarto Privado'
df2.loc[df2.tipo == 'Entire home/apt', ['tipo']] = 'Casa/apt Inteiro'
df2.loc[df2.tipo == 'Shared room', ['tipo']] = 'Quarto Compartilhado'

In [ ]:
df2

,realSum,tipo,compartilhado,privado,p_capacidade,superhost,multi,biz,nota_limpeza,media_satisfacao,quartos,dist,metro_dist,attr_index,attr_index_norm,rest_index,rest_index_norm,lng,lat
0,194.033698,Quarto Privado,False,True,2.0,False,1,0,10.0,93.0,1,5.022964,2.539380,78.690379,4.166708,98.253896,6.846473,4.90569,52.41772
1,344.245776,Quarto Privado,False,True,4.0,False,0,0,8.0,85.0,1,0.488389,0.239404,631.176378,33.421209,837.280757,58.342928,4.90005,52.37432
2,264.101422,Quarto Privado,False,True,2.0,False,0,1,9.0,87.0,1,5.748312,3.651621,75.275877,3.985908,95.386955,6.646700,4.97512,52.36103
3,433.529398,Quarto Privado,False,True,4.0,False,0,1,9.0,90.0,2,0.384862,0.439876,493.272534,26.119108,875.033098,60.973565,4.89417,52.37663
4,485.552926,Quarto Privado,False,True,2.0,True,0,0,10.0,98.0,1,0.544738,0.318693,552.830324,29.272733,815.305740,56.811677,4.90051,52.37508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1098,2486.115342,Casa/apt Inteiro,False,False,2.0,False,0,0,10.0,100.0,1,2.375833,1.436054,181.198355,9.594573,225.304203,15.699521,4.85869,52.37677
1099,233.637194,Quarto Privado,False,True,2.0,False,1,0,10.0,90.0,1,4.719736,0.322263,108.024880,5.719989,134.775277,9.391335,4.83611,52.34910
1100,317.062311,Quarto Privado,False,True,2.0,False,1,0,10.0,92.0,1,0.598355,0.751993,440.476515,23.323524,625.947562,43.616927,4.88897,52.37798
1101,1812.855904,Casa/apt Inteiro,False,False,4.0,False,0,0,8.0,84.0,5,1.943020,0.388532,257.948105,13.658524,336.589237,23.454022,4.90688,52.35794


In [ ]:
db = client['tratado']
colecaoairbnb = db['airbnb']

In [ ]:
df2_dict = df2.to_dict('records')

colecaoairbnb.insert_many(df2_dict)

## Alesp

Bruto

In [ ]:
path3 = 'gs://ad1-bucket/brutos/depsp.csv'

In [ ]:
dfalesp = pd.read_csv(path3)

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
db = client['bruto']
colecaoalesp = db['alesp']

In [ ]:
dfalesp_dict = dfalesp.to_dict('records')

colecaoalesp.insert_many(dfalesp_dict)

tratado

In [ ]:
dfalesp

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
0,2015,300257,3,200.00,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
1,2015,300257,3,295.40,68064740000125,ABELARDO CAMARINHA,E - MATERIAIS DE ESCRITÓRIO E OUTROS MATERIAIS...,PAPER FACE COMERCIAL LTDA
2,2015,300257,3,2850.00,22145388877,ABELARDO CAMARINHA,N - MORADIA,LARA SERVINO VARGAS ABREU
3,2015,300257,4,100.00,10915883000113,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,CENTRO AUTOMOTIVO TOP 23 LTDA
4,2015,300257,4,195.01,71806251000106,ABELARDO CAMARINHA,A - COMBUSTÍVEIS E LUBRIFICANTES,AUTO POSTO DE SERVIÇOS MARV LTDA
...,...,...,...,...,...,...,...,...
542390,2007,300441,3,120.00,6306675000140.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",OPTEI EXPRESS LTDA.
542391,2007,300441,3,2096.80,52204617000122.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",CHURRASCARIA JANDA LTDA.
542392,2007,300441,3,13.48,62925474000282.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",ASSOCIAÇÃO FUNC.ASSEMB.LEG. EST.SÃO PAULO
542393,2007,300441,3,38.08,66747577000170.0,ÍTALO CARDOSO,"I - HOSPEDAGEM, ALIMENTAÇÃO E DESPESAS DE LOCO...",IBIZA EUROPEIA ALIMENTOS LTDA


In [ ]:
dfalesp['Fornecedor'].value_counts().head(20)

PEDÁGIO                                                     7689
TAXI                                                        3415
TELEFONICA BRASIL S.A.                                      2903
CLARO S/A                                                   2468
TELEFONICA BRASIL S.A                                       2304
NEXTEL TELECOMUNICAÇÕES LTDA                                2156
VIVO                                                        1974
SABESP                                                      1948
VIVO S.A.                                                   1938
TELEFÔNICA BRASIL S.A.                                      1847
TELEFONICA                                                  1786
COMPANHIA PAULISTA DE FORÇA E LUZ                           1688
TELEFONICA BRASIL S/A                                       1630
WAGNER HUBERT EPP                                           1570
TELESP CELULAR S/A                                          1493
UNIDAS VEÍCULOS ESPECIAIS

In [ ]:
dfalesp.loc[dfalesp['Fornecedor'].str.endswith('GIO')]

,Ano,Matricula,Mes,Valor,CNPJ,Deputado,Tipo,Fornecedor
855,2003,300439,3,192.6,NaN,ADILSON BARROSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
882,2003,300439,4,104.8,NaN,ADILSON BARROSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
920,2003,300439,5,154.8,NaN,ADILSON BARROSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
960,2003,300439,6,12.2,NaN,ADILSON BARROSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
1017,2003,300439,8,7.4,NaN,ADILSON BARROSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
...,...,...,...,...,...,...,...,...
542223,2006,300441,10,80.6,NaN,ÍTALO CARDOSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
542262,2006,300441,11,19.7,NaN,ÍTALO CARDOSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
542299,2006,300441,12,63.2,NaN,ÍTALO CARDOSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO
542326,2007,300441,1,14.6,NaN,ÍTALO CARDOSO,C - MATERIAIS E SERVIÇOS DE MANUT E CONSERV DE...,PEDÁGIO


In [ ]:
dfalespbackup = dfalesp.copy()

In [ ]:
dfalesp = dfalespbackup.copy()

In [ ]:
dfalesp['Fornecedor'] = dfalesp['Fornecedor'].replace({'^PED.*GIO$': 'PEDÁGIO'}, regex=True)

In [ ]:
dfalesp['Fornecedor'].value_counts().head(20)

TELEFONICA BRASIL S.A                                       13730
KALUNGA COM. E IND. GRÁFICA LTDA                             9318
PEDÁGIO                                                      7689
VIVO S.A                                                     4913
CLARO S.A                                                    4566
TAXI                                                         3415
TELESP CELULAR S/A                                           2899
NEXTEL TELECOMUNICAÇÕES LTDA                                 2156
SABESP                                                       1948
COMPANHIA PAULISTA DE FORÇA E LUZ                            1688
WAGNER HUBERT EPP                                            1570
UNIDAS VEÍCULOS ESPECIAIS S.A.                               1403
NET SERVIÇOS DE COMUNICAÇÃO S/A                              1162
ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO PAULO S.A.     1141
TERRA NETWORKS BRASIL S/A                                    1122
UOL       

In [ ]:
substitui = (dfalesp['Fornecedor'].str.startswith('TELEF')) & (dfalesp['Fornecedor'].str.endswith('BRASIL S.A.'))
substitui2 = (dfalesp['Fornecedor'].str.startswith('TELEF')) & (dfalesp['Fornecedor'].str.endswith('NICA'))
substitui3 =(dfalesp['Fornecedor'].str.startswith('TELEF')) & (dfalesp['Fornecedor'].str.endswith('BRASIL SA'))
substitui4 = (dfalesp['Fornecedor'].str.startswith('TELEF')) & (dfalesp['Fornecedor'].str.endswith('BRASIL S/A'))
substitui5 = (dfalesp['Fornecedor'] == 'VIVO')
substitui6 = (dfalesp['Fornecedor'] == 'VIVO S.A.')
substitui7 = (dfalesp['Fornecedor'] == 'CLARO S/A')
substitui8 = (dfalesp['Fornecedor'] == 'CLARO S.A.' )
substitui9 =(dfalesp['Fornecedor'] == 'TELEFÔNICA BRASIL S.A')
substitui10 =(dfalesp['Fornecedor'] == 'VIVO S/A')
substitui11 =(dfalesp['Fornecedor'].str.startswith('TELESP'))
substitui12 = (dfalesp['Fornecedor'].str.startswith('KALUNGA'))

In [ ]:
dfalesp.loc[substitui, 'Fornecedor'] = 'TELEFONICA BRASIL S.A'

In [ ]:
dfalesp.loc[substitui2, 'Fornecedor'] = 'TELEFONICA BRASIL S.A'

In [ ]:
dfalesp.loc[substitui3, 'Fornecedor'] = 'TELEFONICA BRASIL S.A'

In [ ]:
dfalesp.loc[substitui4, 'Fornecedor'] = 'TELEFONICA BRASIL S.A'

In [ ]:
dfalesp.loc[substitui5, 'Fornecedor'] = 'VIVO S.A'

In [ ]:
dfalesp.loc[substitui6, 'Fornecedor'] = 'VIVO S.A'

In [ ]:
dfalesp.loc[substitui7, 'Fornecedor'] = 'CLARO S.A'

In [ ]:
dfalesp.loc[substitui8, 'Fornecedor'] = 'CLARO S.A'

In [ ]:
dfalesp.loc[substitui9, 'Fornecedor'] = 'TELEFONICA BRASIL S.A'

In [ ]:
dfalesp.loc[substitui10, 'Fornecedor'] = 'VIVO S.A'

In [ ]:
dfalesp.loc[substitui11, 'Fornecedor'] = 'TELESP CELULAR S/A'

In [ ]:
dfalesp.loc[substitui12,'Fornecedor'] = 'KALUNGA COM. E IND. GRÁFICA LTDA'

In [ ]:
db = client['tratado']
colecaoalesp = db['alesp']

In [ ]:
dfalesp_dict2 = dfalesp.to_dict('records')

colecaoalesp.insert_many(dfalesp_dict2)